In [1]:
!pip install -U transformers
!pip install accelerate
!pip install datasets
!pip install bitsandbytes
!pip install einops
!pip install peft


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 627.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.8 MB/s eta 0:00:00


In [1]:
from huggingface_hub import notebook_login

notebook_login()

### Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")
dataset

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 112165
    })
})

In [3]:
test_size = 0.2

dataset = dataset["train"].train_test_split(test_size=test_size)
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 89732
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 22433
    })
})

In [4]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [5]:
train_dataset.to_pandas().head(5)

,instruction,input,output
0,"If you are a doctor, please answer the medical...","i am 37 weeks pregnant , due july 12th i was c...",Hi. Chat Doctor. I welcome you to Chat Docto...
1,"If you are a doctor, please answer the medical...",my daughter 5+ year has a stone in common duct...,Hi.noted history of choledocholithiasis (stone...
2,"If you are a doctor, please answer the medical...",I feel a sort of like a narrow canyon or crack...,Hello and welcome. Yes the curved lines or rid...
3,"If you are a doctor, please answer the medical...",Hello I was wondering what your thoughts was a...,"Hello, my name is Chat Doctor. Burning or pain..."
4,"If you are a doctor, please answer the medical...",I keep having severe cramping when I do situps...,Hi. Thanks for your query and elucidate histor...


In [6]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [7]:
def formatting_func(sample):
    text = f"""Context: {sample["instruction"]}\n
    ### Question: {sample["input"]}\n
    ### Answer: {sample["output"]}
    """
    return text

### Tokenizer

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          padding_side="left",
                                          add_eos_token=True,
                                          add_bos_token=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [10]:
train_tokenized = train_dataset.map(generate_and_tokenize_prompt)
test_tokenized = test_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/89732 [00:00<?, ? examples/s]

Map:   0%|          | 0/22433 [00:00<?, ? examples/s]

In [11]:
print(train_tokenized[0]["input_ids"])

[1, 14268, 28747, 1047, 368, 460, 264, 6676, 28725, 4665, 4372, 272, 5714, 4224, 2818, 356, 272, 7749, 28742, 28713, 5436, 28723, 13, 13, 2287, 774, 22478, 28747, 613, 837, 28705, 28770, 28787, 4587, 15446, 1200, 2940, 461, 3089, 28705, 28740, 28750, 362, 613, 403, 9681, 461, 1802, 28705, 28740, 28787, 362, 304, 403, 2240, 613, 837, 28705, 28770, 8372, 14262, 601, 304, 586, 277, 784, 833, 5915, 1433, 2485, 28745, 586, 19146, 659, 7679, 2691, 346, 298, 272, 1305, 369, 613, 506, 264, 5401, 2764, 297, 586, 19146, 28723, 315, 837, 12785, 767, 513, 586, 2130, 1235, 448, 1721, 304, 613, 14262, 380, 298, 28705, 28781, 8372, 1159, 586, 2940, 3608, 28723, 767, 460, 586, 2877, 28804, 13, 28705, 13, 2287, 774, 26307, 28747, 15359, 28723, 28705, 23781, 16505, 28723, 28705, 315, 10058, 368, 298, 23781, 16505, 550, 4599, 28738, 14821, 12296, 775, 1604, 28723, 315, 506, 4214, 1059, 574, 2996, 28725, 304, 315, 1073, 315, 506, 8988, 574, 4368, 28723, 13695, 28725, 315, 622, 3397, 368, 272, 1489, 2572, 

In [12]:
print("Input: " + train_tokenized[0]["input"])

Input: i am 37 weeks pregnant , due july 12th i was checked june 17th and was told i am 3 cm dialated and my cervix recently got short; my belly has dropped completly to the point that i have a huge space in my belly. I am wondering what if my water doesnt break and i dialate to 4 cm before my due date. what are my options?


In [13]:
print("Output: " + train_tokenized[0]["output"])

Output: Hi.  Chat Doctor.  I welcome you to Chat Doctor VIRTUAL CLINIC. I have gone through your question, and I think I have understood your concern. Furthermore, I will suggest you the best possible treatment options. Please do not worry, as you have crossed 3 weeks of pregnancy. Even if you land in labor pains and bag of water doesn't break doctor can do a procedure called ARM, artificial rupture of membrane, and help you out. You keep watch on babies kick counts. Any discharge or reduced baby kick count should alert you. I hope this answer helps you. Thanks.


In [14]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

In [15]:
# plot_data_lengths(train_tokenized, test_tokenized)

In [16]:
max_length = 512 # This was an appropriate max length for my dataset

def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(formatting_func(prompt),
                       truncation=True,
                       max_length=max_length,
                       padding="max_length")

    result["labels"] = result["input_ids"].copy()
    return result

In [17]:
train_tokenized = train_dataset.map(generate_and_tokenize_prompt2)
test_tokenized = test_dataset.map(generate_and_tokenize_prompt2)

Map:   0%|          | 0/89732 [00:00<?, ? examples/s]

Map:   0%|          | 0/22433 [00:00<?, ? examples/s]

### Quantization

In [18]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_4bit=True,
                                bnb_4bit_use_double=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

### Model

In [19]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00009-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00010-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The target location /root/.cache/huggingface/hub only has 4420.12 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 4983.00 MB. The target location /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/blobs only has 4420.12 MB free disk space.
  warnings.warn(


model-00011-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

OSError: ignored

### Evaluate

In [ ]:
eval_prompt = """ Given the following biometric data, score the users' health, from 0-100.

### Biometric Data:
Temperature=98.2,
Sex=F,
Age=29,
Height=69 inches,
Weight=160 lbs,
V02_Max=55,
HRV=55

### Health Score:
"""

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id,
                                               add_bos_token=True)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()

In [ ]:
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input,
                                               max_new_tokens=256)[0],
                                skip_special_tokens=True))

## Lora

In [4]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

NameError: ignored

In [8]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad():
            training_params += param.numel()

    print(f"""trainable params: {trainable_params} || all params: {all_param} ||
            trainable%: {100 * trainable_params / all_param}""")

In [9]:
print(model)

NameError: ignored

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(r=32,
                    lora_alpha=64,
                    target_modules=["q_proj",
                                    "k_proj",
                                    "v_proj",
                                    "o_proj",
                                    "gate_proj",
                                    "up_proj",
                                    "down_proj",
                                    "lm_head"],
                    bias="none",
                    lora_dropout=0.05,  # Conventional
                    task_type="CAUSAL_LM")

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
print(model)

### Accelerator

In [10]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True,
                                               rank0_only=False))

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
model = accelerator.prepare_model(model)

### Trainering Arguments

In [26]:
from transformers import TrainingArguments

output_dir = "FineTune_Mistral_V2_QA"

training_arguments = TrainingArguments(output_dir=output_dir,
                                       warmup_steps=1,
                                       per_device_train_batch_size=2,
                                       per_device_eval_batch_size=2,
                                       gradient_accumulation_steps=1,
                                       gradient_checkpointing=True,
                                       max_steps=500,
                                       num_train_epochs=1,
                                       learning_rate=2.5e-5,          # Want a small lr for finetuning
                                       # bf16=True,
                                       optim="paged_adamw_8bit",
                                       logging_steps=25,              # When to start reporting loss
                                       logging_dir="./logs",          # Directory for storing logs
                                       save_strategy="steps",         # Save the model checkpoint every logging step
                                       save_steps=25,                 # Save checkpoints every 50 steps
                                       evaluation_strategy="steps",   # Evaluate the model every logging step
                                       eval_steps=25,                 # Evaluate and save checkpoints every 50 steps
                                       do_eval=True,                  # Perform evaluation at the end of training
                                       push_to_hub=True)

### Data Collator

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Trainer

In [21]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
from time import pref_counter
from transformers import Trainer


trainer = Trainer(model=model,
                  train_dataset=train_tokenized,
                  eval_dataset=test_tokenized,
                  args=training_arguments,
                  data_collator=data_collator)
model.config.use_cache = False

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

###Evaluate

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id,
                                               add_bos_token=True,
                                               trust_remote_code=True)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "FineTune_Mistral_V2_QA/checkpoint-01")

In [ ]:
eval_prompt = """ Given the following biometric data, score the users' health, from 0-100.

### Biometric Data:
Temperature=98.2,
Sex=F,
Age=29,
Height=69 inches,
Weight=160 lbs,
V02_Max=55,
HRV=55

### Health Score:
"""

In [ ]:
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True))